# Copernicus Sentinel-2 Data Search and Exploration

This Jupyter Notebook demonstrates how to search, filter, and explore Sentinel-2 satellite imagery metadata using the `phidown` library. The workflow includes:

- **Setting Search Parameters:**  
    The notebook defines parameters such as collection name (`SENTINEL-2`), product type (`S2MSI1C`), cloud cover threshold, area of interest (AOI) in WKT format, date range, and sorting options.

- **Executing the Search:**  
    An instance of `CopernicusDataSearcher` is configured with the specified parameters and used to query the Copernicus Open Access Hub for matching Sentinel-2 products.

- **Viewing Results:**  
    The resulting metadata is loaded into a pandas DataFrame (`df`). Key columns such as `Id`, `Name`, `S3Path`, and `GeoFootprint` are displayed for quick inspection.

## Variables and Data

- **aoi_wkt:** AOI polygon in WKT format. Polygon must start and end with the same point. Coordinates must be given in EPSG 4326
- **cloud_cover_threshold:** Maximum allowed cloud cover percentage.
- **collection_name:** Data collection to search (`SENTINEL-2`).
- **columns_to_display:** List of DataFrame columns to display.
- **df:** DataFrame containing search results (1000 entries, 16 columns).
- **searcher:** Configured `CopernicusDataSearcher` instance.
- **start_date, end_date:** Date range for the search.
- **product_type, orbit_direction, order_by, top:** Additional search parameters.

In [ ]:
%load_ext autoreload
%autoreload 2

# Search

You can modify the search parameters to refine your query, re-run the search, and further analyze or visualize the results as needed.

### Sentinel Products

In [8]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

aoi_wkt = "POLYGON((-95.0 29.0, -95.0 30.0, -94.0 30.0, -94.0 29.0, -95.0 29.0))"  # Define the AOI in WKT format
aoi_wkt = None 

searcher = CopernicusDataSearcher()
# Configure the search parameters
searcher.query_by_filter(
    collection_name='SENTINEL-1',
    product_type=None,
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=aoi_wkt,  # Example: aoi_wkt=aoi_wkt if you want to use the defined AOI
    start_date = '2023-05-03T00:00:00',
    end_date = '2024-05-03T04:00:00',
    top=1000,
    count=True,  # Set to True to get the total count of results
    attributes={'processingLevel':'LEVEL0',
                'operationalMode': 'SM',
                # 'swathIdentifier': 'S1', # Swath identifier is: 1,2,3,4,5,6 for RAW
                } # Swath identifier Stripmap: S1, ..., S6 | 'WV1,WV2' for wavemode
    # base_url defaults to 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products'
    # order_by defaults to 'ContentDate/Start desc'
)

df = searcher.execute_query()
print(f"Number of results: {len(df)}")
# Display the first few rows of the DataFrame
searcher.display_results(top_n=5)

Number of results: 910


,Id,Name,S3Path,GeoFootprint,OriginDate,Attributes
0,00cd06ea-80e6-4ee2-9fb3-b5c0ec189186,S1A_S6_RAW__0SDV_20240502T195132_20240502T1951...,/eodata/Sentinel-1/SAR/S6_RAW__0S/2024/05/02/S...,"{'type': 'Polygon', 'coordinates': [[[-24.5551...",2024-05-02 20:15:12,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,e68c7e10-7213-443e-857c-f9ddadb648fb,S1A_S4_RAW__0SDV_20240502T193657_20240502T1937...,/eodata/Sentinel-1/SAR/S4_RAW__0S/2024/05/02/S...,"{'type': 'Polygon', 'coordinates': [[[-12.4077...",2024-05-02 20:15:29,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,8f52959c-83d4-4568-8b9c-3bdc7933c14b,S1A_S1_RAW__0SDH_20240502T121147_20240502T1212...,/eodata/Sentinel-1/SAR/S1_RAW__0S/2024/05/02/S...,"{'type': 'Polygon', 'coordinates': [[[-88.2859...",2024-05-02 13:07:28,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
3,54192b79-3683-4234-82f6-78f154af2d1c,S1A_S4_RAW__0SDV_20240502T062925_20240502T0629...,/eodata/Sentinel-1/SAR/S4_RAW__0S/2024/05/02/S...,"{'type': 'Polygon', 'coordinates': [[[-177.858...",2024-05-02 07:16:16,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
4,c03e2735-12b5-47cd-9deb-af959af4aa55,S1A_S6_RAW__0SDV_20240502T055859_20240502T0559...,/eodata/Sentinel-1/SAR/S6_RAW__0S/2024/05/02/S...,"{'type': 'Polygon', 'coordinates': [[[-12.6051...",2024-05-02 07:16:24,"[{'@odata.type': '#OData.CSC.StringAttribute',..."


### CCM

In [ ]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

searcher = CopernicusDataSearcher()
# Configure the search parameters
searcher.query_by_filter(
    collection_name='CCM',
    product_type='NAO_MS4_2A_07B6',
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=None,  # Use the defined AOI
    start_date='2014-05-03T00:00:00',
    end_date='2024-05-03T04:00:00',
    top=1000,
    attributes=None
)

df = searcher.execute_query()
print(f"Number of results: {len(df)}")
# Display the first few rows of the DataFrame
searcher.display_results(top_n=5)

### Search by Product Name (Exact Match)

You can search for a specific product if you know its exact name.

**Tip:**

To search by exact product name, make sure the name exists in the Copernicus Data Space catalogue. You can find valid product names by running a general search first, or by browsing the [Copernicus Data Space Catalogue](https://dataspace.copernicus.eu/browser/). If your search returns no results, try using a product name from the official documentation or a recent search result.

Below, we use a known-valid product name as an example.

In [2]:
from phidown.search import CopernicusDataSearcher

searcher_by_name = CopernicusDataSearcher()

# Replace with an actual product name you want to find
product_to_find = 'S1A_IW_GRDH_1SDV_20141031T161924_20141031T161949_003076_003856_634E.SAFE' # Example, replace with a recent, valid name

print(f"Searching for product with exact name: {product_to_find}\n")
df_exact = searcher_by_name.query_by_name(product_name=product_to_find)

if not df_exact.empty:
    searcher_by_name.display_results(top_n=1)
    display(df_exact)
else:
    print(f"Product '{product_to_find}' not found or an error occurred.")

Searching for product with exact name: S1A_IW_GRDH_1SDV_20141031T161924_20141031T161949_003076_003856_634E.SAFE



,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,Attributes
0,application/octet-stream,c23d5ffd-bc2a-54c1-a2cf-e2dc18bc945f,S1A_IW_GRDH_1SDV_20141031T161924_20141031T1619...,application/octet-stream,1748087439,2014-12-27 02:54:17,2016-08-21T07:27:38.211866Z,2024-06-11T19:29:45.682674Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/GRD/2014/10/31/S1A_IW_G...,"[{'Value': '2532e4332148e698b1d07669a8cd9ca0',...","{'Start': '2014-10-31T16:19:24.220594Z', 'End'...",geography'SRID=4326;POLYGON ((19.165325 54.983...,"{'type': 'Polygon', 'coordinates': [[[19.16532...","[{'@odata.type': '#OData.CSC.StringAttribute',..."


## 📥 Download Your Product

Important:
If your credentials are not deleted automatically, you must manually revoke them here:

🔗 Manage S3 Credentials: https://eodata-s3keysmanager.dataspace.copernicus.eu/panel/s3-credentials

Stay secure — always clean up your credentials!

In [1]:
from phidown.search import CopernicusDataSearcher

searcher = CopernicusDataSearcher() 

filename = 'S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE'
searcher.download_product(filename, output_dir='.')

Output directory: /Users/robertodelprete/Desktop/PyProjects/sar/phidown
Pulling down from S3 path: /eodata/Sentinel-1/SAR/S3_SLC__1S/2024/04/30/S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE


INFO:phidown.s5cmd_utils:Created configuration file: .s5cfg
INFO:phidown.s5cmd_utils:Downloading from: s3://eodata/Sentinel-1/SAR/S3_SLC__1S/2024/04/30/S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE/*
INFO:phidown.s5cmd_utils:Output directory: /Users/robertodelprete/Desktop/PyProjects/sar/phidown/S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE
INFO:phidown.s5cmd_utils:Running command: s5cmd --endpoint-url https://eodata.dataspace.copernicus.eu cp s3://eodata/Sentinel-1/SAR/S3_SLC__1S/2024/04/30/S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE/* /Users/robertodelprete/Desktop/PyProjects/sar/phidown/S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE/
